In [1]:
import time
import cv2
import numpy as np

image = cv2.imread('r3.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(gray, (5, 5), 0)
_, threshold = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('Gray', threshold)
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower_white = np.array([0, 0, 200])  # Adjust values based on your image
upper_white = np.array([180, 50, 255])
mask = cv2.inRange(hsv, lower_white, upper_white)
pill_contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


for contour in contours:

    (x, y), radius = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    radius = int(radius)
    
    if radius > 30 and radius < 200:

        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        if circularity > 0.1:
            cv2.circle(image, center, radius, (0, 255, 0), 2)
            cv2.drawContours(image, [contour], 0, (0, 255, 0), 2)
            cv2.putText(image, f"Radius:{radius} pixels", (int(x), int(y)),cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(image, f"Circularity:{circularity*100:.2f}%", (int(x), int(y+20)),cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
            
        
            check = True
            final_dist = 1000
            final_center = 0
            final_radius = 0
            for pill in pill_contours:
                (px, py), pill_radius = cv2.minEnclosingCircle(pill)
                if pill_radius > 15 and pill_radius < 25:
                    pill_center = (int(px), int(py))
                    pill_radius = int(pill_radius)
                    
                    distance = np.sqrt((x - px) ** 2 + (y - py) ** 2)
                    if distance < final_dist:
                        final_dist = distance
                        final_center = pill_center
                        final_radius = pill_radius
                        
            print(final_dist, radius * 0.1)            
            if final_dist < radius * 0.1:
                cv2.circle(image, final_center, final_radius, (255, 0, 0), 2)
                cv2.putText(image, "Pill", (int(px), int(py)),
                            cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
            else:
                if check:
                    cv2.circle(image, final_center, final_radius, (255, 0, 0), 2)
                    cv2.putText(image, f"Misaligned Pill", (int(x), int(y)+40),
                                cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(image, f"({final_dist:.2f})", (int(x), int(y)+60),
                                cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
                    check = False

cv2.imshow('Result', image)
cv2.imwrite("result.jpg", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1000 3.4000000000000004


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Input argument doesn't provide sequence protocol
>  - Can't parse 'center'. Input argument doesn't provide sequence protocol
